In [1]:
import h5py
import torch
import shutil
import os 

In [2]:
def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File(fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best,task_id, filename='checkpoint.pth.tar'):
    model_name = task_id + "model_best.pth.tar"
    # model_name = task_id + "model_best_hmn.pth.tar"
    base_path = 'D:/Dev/project/roofcounting/'
#     base_path = '/home/jthe/Dev/project/roofcounting/'
#     base_path = ('/content/drive/MyDrive/CSRNet-pytorch/')  
    torch.save(state, task_id+filename)
    if is_best:
        shutil.copyfile(task_id+filename, os.path.join(base_path,model_name))            

In [3]:
from torch import nn
import torch
from torchvision import models


class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
#            for i in xrange(len(self.frontend.state_dict().items())):
            for i in range(len(self.frontend.state_dict().items())):
#                self.frontend.state_dict().items()[i][1].data[:] = mod.state_dict().items()[i][1].data[:]
                list(self.frontend.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i][1].data[:] 
    
    def forward(self,x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
                
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)                


In [4]:
import random
import os
from PIL import Image,ImageFilter,ImageDraw
import numpy as np
import h5py
from PIL import ImageStat
import cv2

def load_data(img_path,train = True):
    gt_path = img_path.replace('.png','.h5').replace('images','ground_truth')
    img = Image.open(img_path).convert('RGB')
    gt_file = h5py.File(gt_path)
    target = np.asarray(gt_file['density'])
    if False:
        crop_size = (img.size[0]/2,img.size[1]/2)
        if random.randint(0,9)<= -1:
            
            
            dx = int(random.randint(0,1)*img.size[0]*1./2)
            dy = int(random.randint(0,1)*img.size[1]*1./2)
        else:
            dx = int(random.random()*img.size[0]*1./2)
            dy = int(random.random()*img.size[1]*1./2)
        
        
        
        img = img.crop((dx,dy,crop_size[0]+dx,crop_size[1]+dy))
        target = target[dy:crop_size[1]+dy,dx:crop_size[0]+dx]
        
        
        
        
        if random.random()>0.8:
            target = np.fliplr(target)
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
    
    
    
    target = cv2.resize(target,(target.shape[1]//8,target.shape[0]//8),interpolation = cv2.INTER_CUBIC)*64
    
    
    return img,target


In [5]:
import os
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
# from image import *
import torchvision.transforms.functional as F

class listDataset(Dataset):
    def __init__(self, root, shape=None, shuffle=True, transform=None,  train=False, seen=0, batch_size=1, num_workers=4):
        if train:
            root = root *4
        random.shuffle(root)
        
        self.nSamples = len(root)
        self.lines = root
        self.transform = transform
        self.train = train
        self.shape = shape
        self.seen = seen
        self.batch_size = batch_size
        self.num_workers = num_workers
        
        
    def __len__(self):
        return self.nSamples
    def __getitem__(self, index):
        assert index <= len(self), 'index range error' 
        
        img_path = self.lines[index]
        
        img,target = load_data(img_path,self.train)
           
        
        if self.transform is not None:
            img = self.transform(img)
        return img,target

In [6]:
import sys
import os
import warnings

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

import numpy as np
import json
import cv2
import time

import gc

gpu = "0"
# gpu = ""
best_prec1 = 1e6
original_lr = 1e-7
lr = 1e-7
batch_size = 1
momentum = 0.95
decay = 5*1e-4
start_epoch = 0
epochs = 400
steps = [-1, 1, 100, 150]
scales = [1, 1, 1, 1]
workers = 4
seed = time.time()
print_freq = 30
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
torch.cuda.manual_seed(seed)

model = CSRNet()

if gpu != "":
  model = model.cuda()
  criterion = nn.MSELoss(size_average=False).cuda()
  
else:
  model = model.cpu()
  criterion = nn.MSELoss(size_average=False).cpu()

optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)


C:\Users\jthe\anaconda3\envs\roofcount\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jthe\anaconda3\envs\roofcount\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\jthe\anaconda3\envs\roofcount\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [7]:
def train(train_list, model, criterion, optimizer, epoch):
    
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    
    
    train_loader = torch.utils.data.DataLoader(
        listDataset(train_list,
                       shuffle=True,
                       transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]), 
                       train=True, 
                       seen=model.seen,
                       batch_size=batch_size,
                       num_workers=workers),
        batch_size=batch_size)
    print('epoch %d, processed %d samples, lr %.10f' % (epoch, epoch * len(train_loader.dataset), lr))
    
    model.train()
    end = time.time()

    if gpu != "":
      gc.collect()

    for i,(img, target)in enumerate(train_loader):
        if gpu != "":
          torch.cuda.empty_cache()
        # print("i: {}".format(i))
        data_time.update(time.time() - end)
        
        if gpu != "":
          img = img.cuda()
        else:
          img = img.cpu()

        img = Variable(img)
        output = model(img)
        
        
        
        if gpu != "":
          target = target.type(torch.FloatTensor).unsqueeze(0).cuda()
        else:
          target = target.type(torch.FloatTensor).unsqueeze(0).cpu()
          
        target = Variable(target)
        
        
        loss = criterion(output, target)
        
        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  .format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses))
    
def validate(val_list, model, criterion):
    print ('begin test')
    print("clearing gpu cache")
    
    if gpu != "":
      torch.cuda.empty_cache() 

    test_loader = torch.utils.data.DataLoader(
    listDataset(val_list,
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]),  train=False),
    batch_size=batch_size)    
    
    model.eval()
    
    mae = 0
    
    for i,(img, target) in enumerate(test_loader):
        img = img.cuda()
        img = Variable(img)
        output = model(img)
        
        mae += abs(output.data.sum()-target.sum().type(torch.FloatTensor).cuda())
        
    mae = mae/len(test_loader)    
    print(' * MAE {mae:.3f} '
              .format(mae=mae))

    return mae    
        
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    
    
    lr = original_lr
    
    for i in range(len(steps)):
        
        scale = scales[i] if i < len(scales) else 1
        
        
        if epoch >= steps[i]:
            lr = lr * scale
            if epoch == steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count    

In [8]:
base_path = 'D:/Dev/project/roofcounting/'
# base_path = '/home/jthe/Dev/project/roofcounting/'
train_json_path = os.path.join(base_path, "notebook/dev", "part_A_train.json")
val_json_path = os.path.join(base_path, "notebook/dev", "part_A_with_val.json")

with open(train_json_path, 'r') as outfile:
  train_list = json.load(outfile)

with open(val_json_path, 'r') as outfile:
  val_list = json.load(outfile)


In [9]:
task = "0"

pre = 1 # swap this with 0 to train from scratch, 1 for using pre-trained model
pre_trained_path = "0model_best.pth.tar"

pre_file = os.path.join(base_path, pre_trained_path)
if pre:
  if os.path.isfile(pre_file):
    print("=> loading checkpoint '{}'".format(pre_file))

    if gpu != "":
      checkpoint = torch.load(pre_file)
    else:
      checkpoint = torch.load(pre_file, map_location=torch.device("cpu"))
      
    
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}'".format(pre_file, checkpoint['epoch']))

  else:
    print("=> no checkpoint found at '{}'".format(pre_file))


for epoch in range(start_epoch, epochs):
  adjust_learning_rate(optimizer, epoch)

  train(train_list, model, criterion, optimizer, epoch)
  prec1 = validate(val_list, model, criterion)

  is_best = prec1 < best_prec1
  best_prec1 = min(prec1, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
      'epoch': epoch + 1, 
      'arch': None, 
      'state_dict' : model.state_dict(),
      'best_prec1' : best_prec1,
      'optimizer' : optimizer.state_dict(),
  }, is_best, task)

=> loading checkpoint 'D:/Dev/project/roofcounting/0model_best.pth.tar'
=> loaded checkpoint 'D:/Dev/project/roofcounting/0model_best.pth.tar'
epoch 283, processed 110936 samples, lr 0.0000001000
Epoch: [283][0/392]	Time 0.838 (0.838)	Data 0.129 (0.129)	Loss 1.6082 (1.6082)	
Epoch: [283][30/392]	Time 0.283 (0.291)	Data 0.173 (0.167)	Loss 11.4621 (16.6953)	
Epoch: [283][60/392]	Time 0.268 (0.290)	Data 0.173 (0.175)	Loss 9.3352 (18.3739)	
Epoch: [283][90/392]	Time 0.302 (0.288)	Data 0.190 (0.176)	Loss 3.9271 (16.1879)	
Epoch: [283][120/392]	Time 0.311 (0.292)	Data 0.186 (0.180)	Loss 48.2302 (14.6613)	
Epoch: [283][150/392]	Time 0.353 (0.294)	Data 0.229 (0.182)	Loss 0.1314 (13.4480)	
Epoch: [283][180/392]	Time 0.339 (0.297)	Data 0.229 (0.185)	Loss 2.9796 (12.7375)	
Epoch: [283][210/392]	Time 0.331 (0.300)	Data 0.206 (0.187)	Loss 2.2167 (12.0430)	
Epoch: [283][240/392]	Time 0.306 (0.298)	Data 0.181 (0.186)	Loss 0.7582 (11.6381)	
Epoch: [283][270/392]	Time 0.327 (0.300)	Data 0.202 (0.188)	L

Epoch: [289][120/392]	Time 0.267 (0.295)	Data 0.173 (0.188)	Loss 7.2571 (15.8248)	
Epoch: [289][150/392]	Time 0.316 (0.294)	Data 0.205 (0.187)	Loss 12.3114 (14.5881)	
Epoch: [289][180/392]	Time 0.283 (0.294)	Data 0.189 (0.188)	Loss 2.6487 (13.3038)	
Epoch: [289][210/392]	Time 0.316 (0.295)	Data 0.206 (0.188)	Loss 1.9129 (13.0917)	
Epoch: [289][240/392]	Time 0.298 (0.295)	Data 0.188 (0.188)	Loss 0.1035 (12.7778)	
Epoch: [289][270/392]	Time 0.298 (0.295)	Data 0.188 (0.187)	Loss 37.7899 (12.5556)	
Epoch: [289][300/392]	Time 0.294 (0.294)	Data 0.189 (0.187)	Loss 34.4741 (12.7555)	
Epoch: [289][330/392]	Time 0.300 (0.295)	Data 0.188 (0.187)	Loss 0.5541 (12.7556)	
Epoch: [289][360/392]	Time 0.283 (0.294)	Data 0.189 (0.187)	Loss 16.0715 (12.1924)	
Epoch: [289][390/392]	Time 0.299 (0.294)	Data 0.188 (0.187)	Loss 11.9091 (11.9029)	
begin test
clearing gpu cache
 * MAE 19.639 
 * best MAE 15.481 
epoch 290, processed 113680 samples, lr 0.0000001000
Epoch: [290][0/392]	Time 0.246 (0.246)	Data 0.1

Epoch: [295][330/392]	Time 0.316 (0.303)	Data 0.204 (0.195)	Loss 11.8669 (10.2335)	
Epoch: [295][360/392]	Time 0.284 (0.303)	Data 0.190 (0.195)	Loss 0.1398 (10.1625)	
Epoch: [295][390/392]	Time 0.315 (0.303)	Data 0.193 (0.195)	Loss 38.5846 (10.2219)	
begin test
clearing gpu cache
 * MAE 15.535 
 * best MAE 14.848 
epoch 296, processed 116032 samples, lr 0.0000001000
Epoch: [296][0/392]	Time 0.229 (0.229)	Data 0.119 (0.119)	Loss 15.0019 (15.0019)	
Epoch: [296][30/392]	Time 0.299 (0.298)	Data 0.189 (0.193)	Loss 6.1781 (7.8615)	
Epoch: [296][60/392]	Time 0.333 (0.301)	Data 0.189 (0.193)	Loss 0.0274 (7.7579)	
Epoch: [296][90/392]	Time 0.315 (0.301)	Data 0.189 (0.193)	Loss 2.7093 (9.0262)	
Epoch: [296][120/392]	Time 0.299 (0.301)	Data 0.204 (0.192)	Loss 0.5047 (9.3907)	
Epoch: [296][150/392]	Time 0.300 (0.301)	Data 0.188 (0.192)	Loss 3.0674 (9.2629)	
Epoch: [296][180/392]	Time 0.299 (0.301)	Data 0.190 (0.192)	Loss 11.6952 (8.7312)	
Epoch: [296][210/392]	Time 0.315 (0.301)	Data 0.195 (0.192)

Epoch: [302][90/392]	Time 0.284 (0.303)	Data 0.189 (0.193)	Loss 6.9066 (9.2324)	
Epoch: [302][120/392]	Time 0.300 (0.303)	Data 0.191 (0.194)	Loss 1.1907 (8.5568)	
Epoch: [302][150/392]	Time 0.300 (0.302)	Data 0.190 (0.193)	Loss 0.4094 (8.9462)	
Epoch: [302][180/392]	Time 0.299 (0.303)	Data 0.189 (0.194)	Loss 5.5969 (8.6683)	
Epoch: [302][210/392]	Time 0.314 (0.303)	Data 0.204 (0.194)	Loss 4.3514 (9.1076)	
Epoch: [302][240/392]	Time 0.300 (0.303)	Data 0.190 (0.194)	Loss 13.0076 (9.5159)	
Epoch: [302][270/392]	Time 0.300 (0.303)	Data 0.190 (0.194)	Loss 76.8819 (9.7081)	
Epoch: [302][300/392]	Time 0.299 (0.303)	Data 0.205 (0.194)	Loss 18.5560 (9.6220)	
Epoch: [302][330/392]	Time 0.300 (0.303)	Data 0.189 (0.194)	Loss 1.4780 (9.3765)	
Epoch: [302][360/392]	Time 0.316 (0.303)	Data 0.204 (0.194)	Loss 8.8673 (9.1463)	
Epoch: [302][390/392]	Time 0.298 (0.303)	Data 0.189 (0.194)	Loss 1.2963 (9.2580)	
begin test
clearing gpu cache
 * MAE 16.453 
 * best MAE 14.848 
epoch 303, processed 118776 sam

Epoch: [308][330/392]	Time 0.299 (0.302)	Data 0.189 (0.194)	Loss 7.6326 (8.0941)	
Epoch: [308][360/392]	Time 0.299 (0.302)	Data 0.189 (0.194)	Loss 1.6990 (8.3901)	
Epoch: [308][390/392]	Time 0.313 (0.303)	Data 0.204 (0.194)	Loss 11.2771 (8.4812)	
begin test
clearing gpu cache
 * MAE 25.587 
 * best MAE 14.848 
epoch 309, processed 121128 samples, lr 0.0000001000
Epoch: [309][0/392]	Time 0.220 (0.220)	Data 0.115 (0.115)	Loss 0.4680 (0.4680)	
Epoch: [309][30/392]	Time 0.301 (0.300)	Data 0.206 (0.194)	Loss 46.7754 (8.6367)	
Epoch: [309][60/392]	Time 0.316 (0.302)	Data 0.191 (0.193)	Loss 4.7574 (8.9449)	
Epoch: [309][90/392]	Time 0.302 (0.303)	Data 0.191 (0.193)	Loss 4.8110 (9.2549)	
Epoch: [309][120/392]	Time 0.299 (0.303)	Data 0.189 (0.193)	Loss 0.2939 (8.2355)	
Epoch: [309][150/392]	Time 0.298 (0.303)	Data 0.188 (0.193)	Loss 1.6625 (8.4104)	
Epoch: [309][180/392]	Time 0.298 (0.302)	Data 0.204 (0.193)	Loss 8.4589 (8.1071)	
Epoch: [309][210/392]	Time 0.299 (0.302)	Data 0.189 (0.193)	Loss 

...

Epoch: [373][360/392]	Time 0.344 (0.330)	Data 0.218 (0.209)	Loss 3.9598 (4.9946)	
Epoch: [373][390/392]	Time 0.330 (0.330)	Data 0.205 (0.209)	Loss 11.1285 (4.8926)	
begin test
clearing gpu cache
 * MAE 14.993 
 * best MAE 13.764 
epoch 374, processed 146608 samples, lr 0.0000001000
Epoch: [374][0/392]	Time 0.378 (0.378)	Data 0.260 (0.260)	Loss 1.0924 (1.0924)	
Epoch: [374][30/392]	Time 0.346 (0.329)	Data 0.223 (0.209)	Loss 1.2664 (5.0782)	
Epoch: [374][60/392]	Time 0.330 (0.329)	Data 0.204 (0.209)	Loss 3.9273 (4.1496)	
Epoch: [374][90/392]	Time 0.331 (0.329)	Data 0.211 (0.208)	Loss 0.3711 (3.8562)	
Epoch: [374][120/392]	Time 0.327 (0.329)	Data 0.204 (0.209)	Loss 2.6485 (4.4565)	
Epoch: [374][150/392]	Time 0.314 (0.328)	Data 0.204 (0.208)	Loss 0.5834 (4.5527)	
Epoch: [374][180/392]	Time 0.330 (0.328)	Data 0.204 (0.208)	Loss 16.1024 (4.6662)	
Epoch: [374][210/392]	Time 0.331 (0.328)	Data 0.204 (0.208)	Loss 0.9398 (4.5642)	
Epoch: [374][240/392]	Time 0.314 (0.328)	Data 0.205 (0.207)	Loss 

Epoch: [380][150/392]	Time 0.331 (0.327)	Data 0.208 (0.207)	Loss 9.8075 (4.3402)	
Epoch: [380][180/392]	Time 0.330 (0.327)	Data 0.220 (0.207)	Loss 1.2841 (4.3643)	
Epoch: [380][210/392]	Time 0.315 (0.327)	Data 0.205 (0.207)	Loss 15.2506 (4.2343)	
Epoch: [380][240/392]	Time 0.325 (0.327)	Data 0.200 (0.207)	Loss 0.1357 (4.4087)	
Epoch: [380][270/392]	Time 0.330 (0.327)	Data 0.204 (0.207)	Loss 8.5681 (4.4003)	
Epoch: [380][300/392]	Time 0.331 (0.328)	Data 0.220 (0.208)	Loss 8.6459 (4.4995)	
Epoch: [380][330/392]	Time 0.330 (0.328)	Data 0.204 (0.208)	Loss 3.9976 (4.4888)	
Epoch: [380][360/392]	Time 0.314 (0.328)	Data 0.204 (0.208)	Loss 4.3331 (4.4461)	
Epoch: [380][390/392]	Time 0.362 (0.329)	Data 0.205 (0.208)	Loss 11.4096 (4.5531)	
begin test
clearing gpu cache
 * MAE 13.795 
 * best MAE 13.764 
epoch 381, processed 149352 samples, lr 0.0000001000
Epoch: [381][0/392]	Time 0.394 (0.394)	Data 0.268 (0.268)	Loss 0.2353 (0.2353)	
Epoch: [381][30/392]	Time 0.315 (0.330)	Data 0.205 (0.209)	Los

Epoch: [386][390/392]	Time 0.322 (0.329)	Data 0.201 (0.209)	Loss 1.6372 (4.5154)	
begin test
clearing gpu cache
 * MAE 16.565 
 * best MAE 13.557 
epoch 387, processed 151704 samples, lr 0.0000001000
Epoch: [387][0/392]	Time 0.361 (0.361)	Data 0.251 (0.251)	Loss 0.0465 (0.0465)	
Epoch: [387][30/392]	Time 0.361 (0.331)	Data 0.236 (0.212)	Loss 4.5637 (4.4583)	
Epoch: [387][60/392]	Time 0.330 (0.330)	Data 0.204 (0.211)	Loss 6.3040 (4.9392)	
Epoch: [387][90/392]	Time 0.330 (0.329)	Data 0.220 (0.210)	Loss 0.1036 (5.2431)	
Epoch: [387][120/392]	Time 0.330 (0.329)	Data 0.205 (0.209)	Loss 3.6313 (5.0163)	
Epoch: [387][150/392]	Time 0.330 (0.328)	Data 0.220 (0.209)	Loss 1.0806 (4.8149)	
Epoch: [387][180/392]	Time 0.319 (0.328)	Data 0.209 (0.209)	Loss 0.3199 (4.5606)	
Epoch: [387][210/392]	Time 0.330 (0.328)	Data 0.205 (0.209)	Loss 4.8618 (4.4847)	
Epoch: [387][240/392]	Time 0.330 (0.329)	Data 0.205 (0.209)	Loss 5.8518 (4.5258)	
Epoch: [387][270/392]	Time 0.329 (0.329)	Data 0.203 (0.209)	Loss 1.

Epoch: [393][180/392]	Time 0.330 (0.329)	Data 0.211 (0.208)	Loss 1.6899 (4.1691)	
Epoch: [393][210/392]	Time 0.341 (0.329)	Data 0.215 (0.208)	Loss 2.9570 (4.2436)	
Epoch: [393][240/392]	Time 0.314 (0.328)	Data 0.204 (0.208)	Loss 2.8782 (4.2920)	
Epoch: [393][270/392]	Time 0.330 (0.328)	Data 0.204 (0.208)	Loss 3.1161 (4.3141)	
Epoch: [393][300/392]	Time 0.331 (0.328)	Data 0.205 (0.208)	Loss 8.7681 (4.2559)	
Epoch: [393][330/392]	Time 0.315 (0.328)	Data 0.205 (0.208)	Loss 0.8918 (4.4035)	
Epoch: [393][360/392]	Time 0.329 (0.329)	Data 0.204 (0.208)	Loss 4.0873 (4.4204)	
Epoch: [393][390/392]	Time 0.314 (0.329)	Data 0.204 (0.208)	Loss 1.0552 (4.3972)	
begin test
clearing gpu cache
 * MAE 14.117 
 * best MAE 13.557 
epoch 394, processed 154448 samples, lr 0.0000001000
Epoch: [394][0/392]	Time 0.408 (0.408)	Data 0.282 (0.282)	Loss 5.9690 (5.9690)	
Epoch: [394][30/392]	Time 0.328 (0.328)	Data 0.205 (0.208)	Loss 0.8715 (3.2143)	
Epoch: [394][60/392]	Time 0.330 (0.327)	Data 0.204 (0.207)	Loss 0

begin test
clearing gpu cache
 * MAE 13.896 
 * best MAE 13.385 
